In [1]:
import torch
import torch.nn.functional as F

In [2]:
x = torch.empty(48,1,256,256)

In [8]:
_x = x.permute(1,0,2,3).unsqueeze(0)
F.interpolate(_x,scale_factor=(1/2,1,1), mode="nearest").shape

torch.Size([1, 1, 24, 256, 256])

In [23]:
x.shape[0]*2,x.shape[1:]

(96, torch.Size([1, 256, 256]))

In [27]:
_shape = x.shape
torch.tensor([_shape[0]*2,*_shape[1:]])

tensor([ 96,   1, 256, 256])

In [42]:
from monai.transforms import (
    Compose,
    CropForegroundd,
    EnsureChannelFirstd,
    Lambda,
    LoadImaged,
    ResizeWithPadOrCrop,
    ScaleIntensityRangePercentilesd,
    SqueezeDim,
)
transforms = Compose(
    [
        LoadImaged(keys=["image", "mask"]),
        EnsureChannelFirstd(keys=["image", "mask"]),
        ScaleIntensityRangePercentilesd(
            keys=["image"], lower=0.1, upper=99.9, b_min=0.0, b_max=1.0, clip=False
        ),
        # CropForegroundd(
        #     keys=["image", "mask"],
        #     source_key="mask",
        #     # margin=[0, 128, 128] if args.seq == "T1" else [128, 128, 0],
        #     allow_smaller=True,
        # ),
        Lambda(func=lambda x: x["image"]),
    ]
)

In [45]:
transforms = Compose(
    [
        LoadImaged(keys=["image"]),
        EnsureChannelFirstd(keys=["image"]),
        ScaleIntensityRangePercentilesd(
            keys=["image"], lower=0.1, upper=99.9, b_min=0.0, b_max=1.0, clip=False
        ),
        Lambda(func=lambda x: x["image"]),
    ]
)
path = "/root/aicp-data/IXI_downsampledx2_iacl/IXI002-Guys-0828/T1.nii.gz"
mask_path = "/root/aicp-data/IXI_downsampledx2_iacl/IXI002-Guys-0828/T1_mask.nii.gz"
data = transforms({"image": path})

In [46]:
data.shape

torch.Size([1, 128, 256, 173])

In [9]:
import torch
torch.manual_seed(0)


def _A(x):
    # return radon.A(x)
    # return F.interpolate(
    #     x, scale_factor=factor, mode="nearest", align_corners=False
    # )
    indices = torch.arange(0, x.size(0), factor, device=x.device, dtype=torch.long)
    return x.index_select(0, indices)

def _AT(y):
    # y: [Nsamp, C, H, W] or general [...], factor from outer scope
    # target_shape = (y.shape[0] * factor, *y.shape[1:])  # tuple of ints, safe
    target_shape = img_shape
    x_hr = torch.zeros(target_shape, device=y.device, dtype=y.dtype)
    indices = torch.arange(
        0, target_shape[0], factor, device=y.device, dtype=torch.long
    )
    x_hr.index_copy_(0, indices, y)
    return x_hr

device = 'cuda'  # just keep device logic

img_shape = (179,1,256,256)
factor=2.0
# 随机 x, y 维度需与 A/AT 对应
x = torch.randn(img_shape, device='cuda' if torch.cuda.is_available() else 'cpu', dtype=torch.float32)
y = torch.randn((int(img_shape[0]//factor)+1, *img_shape[1:]), device=x.device, dtype=x.dtype)

Ax = _A(x)
ATy = _AT(y)

lhs = float((Ax.flatten() * y.flatten()).sum().cpu().numpy())
rhs = float((x.flatten() * ATy.flatten()).sum().cpu().numpy())
print("Adjoint test: <A x, y> =", lhs, ", <x, A^T y> =", rhs, ", diff =", abs(lhs-rhs))


Adjoint test: <A x, y> = -2289.725830078125 , <x, A^T y> = -2289.725341796875 , diff = 0.00048828125


In [10]:
ones = torch.ones_like(x).to(x.device)
k = img_shape[0]//4
x0 = torch.zeros(img_shape, device=ones.device)
x0[k:k+1] = 1.0
y0 = _A(x0)
x0_back = _AT(y0)
nonzero_sums = x0_back.abs().sum(dim=(1,2,3))
nonzero_idx = (nonzero_sums > 1e-8).nonzero().squeeze().cpu().numpy()
print("nonzero slice indices in x0_back:", nonzero_idx)
print("nonzero sums values (selected):", nonzero_sums[nonzero_idx].cpu().numpy())

nonzero slice indices in x0_back: 44
nonzero sums values (selected): 65536.0


In [11]:
k = img_shape[0]//4
x0 = torch.zeros(img_shape, device=ones.device)
x0[k:k+1] = 1.0
y0 = _A(x0)
x0_back = _AT(y0)
nonzero_sums = x0_back.abs().sum(dim=(1,2,3))
nonzero_idx = (nonzero_sums > 1e-8).nonzero().squeeze().cpu().numpy()
print("nonzero slice indices in x0_back:", nonzero_idx)
print("nonzero sums values (selected):", nonzero_sums[nonzero_idx].cpu().numpy())


nonzero slice indices in x0_back: 44
nonzero sums values (selected): 65536.0


In [21]:
list(range(2))


[0, 1]